In [ ]:
! pip install openai --upgrade
! pip install langchain --upgrade
! pip install python-dotenv
! pip install srt
! pip install codecs
! pip install tiktoken

In [ ]:
import os
import openai
import langchain as lc
from langchain.document_loaders import SRTLoader
from dotenv import load_dotenv
import srt
import codecs
import tiktoken
from IPython.display import clear_output

# Load environment variables
load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

openai.api_key = openai_api_key

GPT_MODEL_NAME = "gpt-4"

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def get_completion(prompt, model=GPT_MODEL_NAME):
    response = client.chat.completions.create(
        temperature=0, 
        model=GPT_MODEL_NAME, 
        messages=[
            {"role": "system", "content": "You are a helpful assistant with expertise in english language reading and writing. You are an expert copywriter."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def write_to_file(content: str, file_path: str):
    """Write content to a file."""
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

def read_srt_file(file_path: str):
    """Read and parse an SRT file."""
    with codecs.open(file_path, 'r', encoding='utf-8-sig') as f:
        return list(srt.parse(f.read()))

### Audio download from Youtube

In [ ]:
from pytube import YouTube
from pydub import AudioSegment
import subprocess
import os

# Improved: Added error handling and streamlined audio format

def download_video(youtube_link: str, output_filename: str = "stream"):
    """Downloads the video stream from a YouTube video."""
    try:
        youtube = YouTube(youtube_link)
        # Selecting the best audio stream
        video_stream = youtube.streams.first()
        if not video_stream:
            raise Exception("No video stream found in the YouTube video.")

        # Downloading and saving the stream
        video_stream.download(filename=f"{output_filename}.mp4")

    except Exception as e:
        print(f"Error downloading video: {e}")

# YouTube video URL
url = 'https://www.youtube.com/watch?v=LUk7INpXM5M'

download_video(url)

### Extract Audio from Video

In [ ]:
! pip install moviepy

In [ ]:
video_file_name = "stream"

In [ ]:
from moviepy.editor import VideoFileClip

def extract_audio(video_file: str, audio_file: str):
    """Extracts audio from a video file and saves it as an audio file."""
    try:
        with VideoFileClip(video_file) as video:
            audio = video.audio
            audio.write_audiofile(audio_file)
    except Exception as e:
        print(f"Error extracting audio: {e}")

# Usage
extract_audio(f"{video_file_name}.mp4", "output_audio.mp3")


### Whisper Transcription

In [ ]:

! pip uninstall whisper
! pip install git+https://github.com/openai/whisper.git
! pip install pydub 
! pip install ffmpeg
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch

is_cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {is_cuda_available}")


In [ ]:
import whisper
from whisper.utils import get_writer
import re

def split_on_punctuation_or_space(text, max_length):
    """
    Splits the text on punctuation marks first, if any segments are longer than max_length,
    then split on spaces as a fallback.
    """
    punctuated_splits = re.split(r'(?<=[.!?]) +', text)
    final_splits = []
    for segment in punctuated_splits:
        if len(segment) > max_length:
            # Further split long segments at the nearest space to the halfway point
            space_splits = segment.split(' ')
            part = ""
            for word in space_splits:
                if len(part + word) < max_length:
                    part += word + " "
                else:
                    final_splits.append(part.strip())
                    part = word + " "  # Start a new part
            final_splits.append(part.strip())  # Add the last part for this segment
        else:
            final_splits.append(segment)
    return final_splits

def estimate_time_per_word(total_duration, total_words):
    return total_duration / total_words if total_words > 0 else 0

def split_transcript_segments(result, max_segment_duration=10):
    new_segments = []
    for segment in result['segments']:
        segment_duration = segment['end'] - segment['start']
        words = segment['text'].split()
        time_per_word = estimate_time_per_word(segment_duration, len(words))
        
        # Estimate max length of segment text based on duration
        max_words_per_segment = max_segment_duration / time_per_word if time_per_word > 0 else 10
        max_length = int(max_words_per_segment * 10)  # Estimate average word length

        split_texts = split_on_punctuation_or_space(segment['text'], max_length)
        
        current_start_time = segment['start']
        for split_text in split_texts:
            split_words = split_text.split()
            split_duration = len(split_words) * time_per_word
            new_segments.append({
                'start': current_start_time,
                'end': current_start_time + split_duration,
                'text': split_text,
            })
            current_start_time += split_duration
    return new_segments

model = whisper.load_model("medium")
audio = "output_audio.mp3"
result = model.transcribe(audio)

# Apply sophisticated splitting
result['segments'] = split_transcript_segments(result)

output_directory = "./"

# Save as an SRT file
try:
    srt_writer = get_writer("srt", output_directory)
    srt_writer(result, audio)
    print(f"SRT file saved.")
except Exception as e:
    print(f"Error saving SRT file: {e}")


### ReSplit SRT File

In [ ]:
import re
from datetime import timedelta

def parse_srt_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    patterns = re.compile(r'(\d+)\n(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.*?)\n\n', re.DOTALL)
    subtitles = []
    for match in patterns.finditer(content):
        index, start, end, text = match.groups()
        subtitles.append({'start': start, 'end': end, 'text': text})
    return subtitles

def srt_time_to_seconds(srt_time):
    hours, minutes, seconds_milliseconds = srt_time.split(':')
    seconds, milliseconds = seconds_milliseconds.split(',')
    total_seconds = int(hours) * 3600 + int(minutes) * 60 + int(seconds) + int(milliseconds) / 1000
    return total_seconds

def seconds_to_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"


def write_srt_file(subtitles, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for i, subtitle in enumerate(subtitles, start=1):
            file.write(f"{i}\n{subtitle['start']} --> {subtitle['end']}\n{subtitle['text']}\n\n")

# Use the previously defined functions for splitting logic
# Assuming split_on_punctuation_or_space and split_transcript_segments are already defined

def improve_srt_file(input_filename, output_filename):
    subtitles = parse_srt_file(input_filename)
    improved_subtitles = []
    for subtitle in subtitles:
        start_seconds = srt_time_to_seconds(subtitle['start'])
        end_seconds = srt_time_to_seconds(subtitle['end'])
        duration = end_seconds - start_seconds
        text = subtitle['text']
        
        # Assuming an average reading speed or adjust according to your preferences
        max_segment_duration = 10
        split_texts = split_on_punctuation_or_space(text, 60)  # Assuming 60 characters as a rough split
        
        time_per_split = duration / len(split_texts)
        for i, split_text in enumerate(split_texts):
            improved_subtitles.append({
                'start': seconds_to_srt_time(start_seconds + i * time_per_split),
                'end': seconds_to_srt_time(start_seconds + (i + 1) * time_per_split),
                'text': split_text
            })
    
    write_srt_file(improved_subtitles, output_filename)

# Example usage
input_srt_filename = '2024-03-11-Prompt-Engineering-Kariyeri.srt'
output_srt_filename = '2024-03-11-Prompt-Engineering-Kariyeri-2.srt'
improve_srt_file(input_srt_filename, output_srt_filename)


### Creating video chapters

In [ ]:
def process_subtitles(subtitles, model_name: str):
    """Process subtitles and split them into chapters."""
    result, chunk = "", ""
    for sub in subtitles:
        chunk += f"{sub.start}>{sub.end.total_seconds()}\n{sub.content}"
        if num_tokens_from_string(chunk) > 7000:
            result += split_into_chapters(chunk, model_name)
            chunk = ""        
    if chunk:
        result += split_into_chapters(chunk, model_name)
    return result

def split_into_chapters(chunk: str, model_name: str):
    """Split the transcript chunk into chapters."""
    prompt = ("Below is a part of a video transcript. You need to split the video "
              "into five topic chapters. The chapters will be used to navigate in the "
              "larger video timeline to let watchers switch between topics. Read the "
              "entire transcript. Once done reading, split it into chapters. Provide "
              "the list of chapters in this format [HH:MM:SS Chapter Name in Turkish]. Put each "
              "chapter in a separate line in plain text using the transcript language. \n\n" + chunk)
    return get_completion(prompt, model_name)


subtitles = read_srt_file("output_audio.srt")
final_output = process_subtitles(subtitles, GPT_MODEL_NAME)
write_to_file(final_output, 'chapters-iac.txt')

### Creating video summary

In [ ]:
def create_summary(transcript: str, model_name: str):
    """Create a summary of the given transcript."""
    prompt = ("Below is a video transcript. Your goal is to summarize the "
              "entire video. You need to create the shortest summary as possible "
              " that will help a reader understand the information given in the video. "
              "Your summary should be in Turkish.\n\n" + transcript)
    return get_completion(prompt, model_name)

subtitles = read_srt_file("output_audio.srt")
transcript_text = "\n\n".join([f"{sub.content}" for sub in subtitles])
final_output = create_summary(transcript_text, GPT_MODEL_NAME)
write_to_file(final_output, 'summaries.txt')

### Title Creator

In [ ]:
def process_srt_file(file_path: str, model_name: str):
    """Process an SRT file to generate Turkish title alternatives and a summary."""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    prompt = ("Below is a series of summaries created out of different sections "
              "of a video recording. The video is published on YouTube. Provide "
              "10 Turkish title alternatives and a single Turkish summary for the "
              "video. Both title and summary should be inviting and helpful to "
              "watchers.\n\n" + content)
    
    return get_completion(prompt, model_name)

final_output = process_srt_file("summaries.txt", GPT_MODEL_NAME)
with open('title-description.txt', 'w', encoding='utf-8') as f:
    f.write(final_output)